In [ ]:
import pandas as pd

# Load fresh data
df = pd.read_csv("netflix1.csv")

# Normalize titles: strip spaces and convert to lowercase
df['title'] = df['title'].str.strip().str.lower()



In [3]:
# Step 1: Clean the title column (if not already done)
df['title_clean'] = df['title'].str.strip().str.lower()

# Step 2: Count each title
title_counts = df['title_clean'].value_counts()

# Step 3: Filter titles that appear more than once
duplicates = title_counts[title_counts > 1]
print("Titles with duplicates:\n", duplicates)

# Step 4: Show full rows for those duplicate titles (optional)
duplicate_rows = df[df['title_clean'].isin(duplicates.index)]
print("\nFull rows with duplicate titles:\n", duplicate_rows[['title', 'director', 'country']])



Titles with duplicates:
 title_clean
esperando la carroza        2
9-feb                       2
fullmetal alchemist         2
consequences                2
15-aug                      2
death note                  2
sin senos sí hay paraíso    2
love in a puff              2
22-jul                      2
Name: count, dtype: int64

Full rows with duplicate titles:
                          title            director        country
220             love in a puff      Pang Ho-cheung      Hong Kong
393                      9-feb           Not Given       Pakistan
415       esperando la carroza     Alejandro Doria      Argentina
537                      9-feb           Not Given       Pakistan
2590              consequences        Ozan Açıktan         Turkey
2925                    15-aug  Swapnaneel Jayakar          India
3285                    22-jul     Paul Greengrass         Norway
3637       fullmetal alchemist       Fumihiko Sori          Japan
3819                death note        

standardize the date_added column format 

In [4]:
# Step 1: Convert to datetime with dayfirst=True to treat 05/01/21 as 5th Jan 2021
df['date_added'] = pd.to_datetime(df['date_added'], dayfirst=True, errors='coerce')

# Step 2: Convert to desired format dd-mm-yyyy
df['date_added'] = df['date_added'].dt.strftime('%d-%m-%Y')

# Optional: Check output
print(df[['title', 'date_added']].head(15))



                               title  date_added
0               dick johnson is dead  25-09-2021
1                          ganglands  24-09-2021
2                      midnight mass  24-09-2021
3   confessions of an invisible girl  22-09-2021
4                            sankofa  24-09-2021
5      the great british baking show  24-09-2021
6                       the starling  24-09-2021
7    motu patlu in the game of zones  01-05-2021
8                       je suis karl  23-09-2021
9           motu patlu in wonderland  01-05-2021
10    motu patlu: deep sea adventure  01-05-2021
11          motu patlu: mission moon  01-05-2021
12                  99 songs (tamil)  21-05-2021
13       bridgerton - the afterparty  13-07-2021
14     bling empire - the afterparty  12-06-2021


/var/folders/l3/6_1d03v13r3423dpvq4sswyw0000gn/T/ipykernel_2390/3149749445.py:2: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['date_added'] = pd.to_datetime(df['date_added'], dayfirst=True, errors='coerce')


Add year_added from date_added:

In [5]:
# Ensure date_added is in datetime format
df['date_added'] = pd.to_datetime(df['date_added'], format='%d-%m-%Y', errors='coerce')

# Extract the year
df['year_added'] = df['date_added'].dt.year



In [6]:
# Step 9: Clean 'duration' column
df[['duration_int', 'duration_type']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')
df['duration_int'] = df['duration_int'].astype('float')  

# Convert to numeric
df[['duration', 'duration_int', 'duration_type']].head()


,duration,duration_int,duration_type
0,90 min,90.0,min
1,1 Season,1.0,Season
2,1 Season,1.0,Season
3,91 min,91.0,min
4,125 min,125.0,min


In [7]:
# Drop 'title_clean' if no longer needed
df.drop(columns=['title_clean'], inplace=True)



In [8]:
# Step 1: Strip and lower the original title column directly
df['title'] = df['title'].str.strip().str.lower()

# Step 2: Drop duplicates based on cleaned title
df = df.drop_duplicates(subset='title', keep='first')

# Step 3: Check new shape (should be 8782 if working correctly)
print("Final shape after duplicate removal:", df.shape)

# Step 4: Export cleaned CSV
df.to_csv("netflix_cleaned_final.csv", index=False)


Final shape after duplicate removal: (8781, 13)


In [11]:
# Save to CSV with the new column
df.to_csv("netflix_cleaned_with_year.csv", index=False)